# Fremont Residential Market Analysis

### Process:
##### Section 1: Preview and Data Cleaning

##### Section 2: Exploratory Data Analysis

##### Section 3: Statistical Analysis

##### Section 4: Price Prediction

##### Section 5: Data-Driven Suggestions


## Feature Information:
- ID: Unique ID for each single house
- DOM: Data on market
- Area: Area codes in Fremont
- LP: Listing price
- SP: Sold price
- SqFt: Square footage for each single house
- BR: Number of bedroom
- Bth: Number of full-bathroom
- PB: Number of partial-bathroom
- Gar: Garage
- GarSp: Garage space
- YrBlt: Year built
- HOA Fee: Homeowner's association fee
- Freq: Frequency of HOA fee (A: Annually, SA: Semi-annual, M: Monthly, Q:uarterly)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [2]:
df = pd.read_csv('Fremont Sold DE 19-21.csv')

In [3]:
df.head()

,ID,DOM,Area,LP,SP,SqFt,BR,Bth,PB,Gar,GarSp,YrBlt,HOA Fee,Freq
0,BA21916741,14,21101,"$647,700","$750,000",1320,3.0,2.0,0.0,NaN,2.0,1959.0,180.0,A
1,BA21929677,33,21101,"$725,000","$705,000",950,3.0,1.0,0.0,NaN,2.0,1957.0,NaN,NaN
2,ME19036856,72,21101,"$799,000","$765,000",1036,3.0,1.0,0.0,NaN,2.0,1959.0,NaN,NaN
3,BA21915173,128,21101,"$829,000","$812,500",1042,2.0,2.0,0.0,NaN,2.0,1985.0,125.0,M
4,BA22004902,23,21101,"$850,000","$836,000",1169,3.0,2.0,0.0,NaN,2.0,1959.0,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4064 entries, 0 to 4063
Data columns (total 14 columns):
ID         4064 non-null object
DOM        4064 non-null int64
Area       4064 non-null int64
LP         4064 non-null object
SP         4064 non-null object
SqFt       4064 non-null int64
BR         4063 non-null float64
Bth        4063 non-null float64
PB         2953 non-null float64
Gar        3901 non-null object
GarSp      4058 non-null float64
YrBlt      4062 non-null float64
HOA Fee    1009 non-null float64
Freq       896 non-null object
dtypes: float64(6), int64(3), object(5)
memory usage: 444.6+ KB


In [5]:
df.isna().sum()

ID            0
DOM           0
Area          0
LP            0
SP            0
SqFt          0
BR            1
Bth           1
PB         1111
Gar         163
GarSp         6
YrBlt         2
HOA Fee    3055
Freq       3168
dtype: int64

In [7]:
df[df.duplicated()]

,ID,DOM,Area,LP,SP,SqFt,BR,Bth,PB,Gar,GarSp,YrBlt,HOA Fee,Freq


### Data cleaning
- Drop the row with missing value in BR, Bth, and YrBlt columns
- Drop the Freq column since we have the HOA fee
- Replace 'NaN' with 0 in PB, GarSp, HOA fee columns
- Replace 'NaN' with 'Y' in Gar column if GarSp not equal to 0, otherwise replace with 'N'
- Remove '\$' and ',' from LP and SP columns
- Change LP, SP, BR, Bth, PB, GarSp, YrBlt, HOA Fee data type to 'int'

In [8]:
df_clean = df.copy()

In [9]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4064 entries, 0 to 4063
Data columns (total 14 columns):
ID         4064 non-null object
DOM        4064 non-null int64
Area       4064 non-null int64
LP         4064 non-null object
SP         4064 non-null object
SqFt       4064 non-null int64
BR         4063 non-null float64
Bth        4063 non-null float64
PB         2953 non-null float64
Gar        3901 non-null object
GarSp      4058 non-null float64
YrBlt      4062 non-null float64
HOA Fee    1009 non-null float64
Freq       896 non-null object
dtypes: float64(6), int64(3), object(5)
memory usage: 444.6+ KB


In [10]:
df_clean.isna().sum()

ID            0
DOM           0
Area          0
LP            0
SP            0
SqFt          0
BR            1
Bth           1
PB         1111
Gar         163
GarSp         6
YrBlt         2
HOA Fee    3055
Freq       3168
dtype: int64

In [11]:
# Drop the row with missing value in BR, Bth, and YrBlt columns

df_clean.dropna(subset=['BR','Bth','YrBlt'], inplace=True)

In [12]:
# Drop the 'Freq' column

df_clean.drop('Freq', axis=1, inplace=True)

In [13]:
# Replace 'NaN' with 0 in PB, GarSp, HOA fee columns

df_clean.update(df[['PB', 'GarSp', 'HOA Fee']].fillna(0))

In [14]:
# Replace 'NaN' with 'Y' in Gar column if GarSp not equal to 0, otherwise replace with 'N'

def filling_Gar(x):
    Gar = x[0]
    GarSp = x[1]
    
    if pd.isna(Gar):
        if GarSp != 0:
            return 'Y'
        else:
            return 'N'
    else:
        return Gar

In [15]:
df_clean['Gar'] = df_clean[['Gar', 'GarSp']].apply(filling_Gar, axis=1)

In [16]:
# Remove '\$' and ',' from LP and SP columns

df_clean['LP'] = df_clean.LP.apply(lambda x: x.replace('$','').replace(',',''))
df_clean['SP'] = df_clean.SP.apply(lambda x: x.replace('$','').replace(',',''))

In [17]:
# Change data type
col = ['LP', 'SP', 'BR', 'Bth', 'PB', 'GarSp', 'YrBlt', 'HOA Fee']

for i in col:
    df_clean[i] = df_clean[i].astype('int')

In [18]:
df_clean['Gar'] = df_clean.Gar.astype('category')

In [19]:
# Check the data set
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4061 entries, 0 to 4063
Data columns (total 13 columns):
ID         4061 non-null object
DOM        4061 non-null int64
Area       4061 non-null int64
LP         4061 non-null int64
SP         4061 non-null int64
SqFt       4061 non-null int64
BR         4061 non-null int64
Bth        4061 non-null int64
PB         4061 non-null int64
Gar        4061 non-null category
GarSp      4061 non-null int64
YrBlt      4061 non-null int64
HOA Fee    4061 non-null int64
dtypes: category(1), int64(11), object(1)
memory usage: 416.5+ KB


In [20]:
# Store the cleaned data set into a csv file

df_clean.reset_index(drop=True)
df_clean.to_csv('Fremont_Sold_data_cleaned.csv', index=False)